# Score Normalization with unsuperviesd info
-------------

z-norm과 s-norm을 구현하였다.

기존에 다른 데이터셋으로 부터 imposter set을 가져와서 score normalization 했던 것에서 벗어나,  
Unsupervised하게 모인 데이터를 가지고 score normalization을 수행

adapt trial의 prediction 기준으로 imposter_embeds를 설정하면 에러 때문에(에러가 꽤 클 경우), imposter set의 score 분포가 깨진다.  
adapte_label을 기준으로 하면 성능이 좋다.

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os, sys
import pickle
import pandas as pd
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from adaptive_sv_system.score_norm_utils import key2df

keys = pickle.load(open("./xvector_embeds/sv_keys.pkl", "rb"))
key_df = key2df(keys)
key2id = {k:v for v, k in enumerate(keys)}

sv_embeds = np.load("./xvector_embeds/sv_embeds.npy")
validation_set = pd.read_pickle("trials/enr306/validation_set.pkl")

In [3]:
def uttrkey_to_id(key2id, keys):
    return [key2id[key] for key in keys]

In [4]:
# from score_norm_utils import uttrkey_to_id
imposter_set = validation_set.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=5))
ood_imposter_embeds = sv_embeds[uttrkey_to_id(key2id, imposter_set.index)]

## Parsing trace then score normalization

TODO: run_trial에 넣어야한다... 너무 느리다.

trace = [adapt_tr, test_tr, ood_tr]  
tr = trial_idxs, scores, label, pred 또는   
tr = trial_idxs, scores, label, pred, enroll_pred  

In [5]:
# threshold = 0.56792
threshold = 0.6292

In [35]:
base_dir = "./adaptive_sv_system/test/FS7_ENR1_QUE7/inc/"
traces = pickle.load(open(base_dir+"trace.pkl", "rb"))
meta_info = pd.read_pickle(base_dir+"meta_info_df.pkl")

In [40]:
import torch
from adaptive_sv_system.score_norm_utils import eval_cosine_score, eval_z_score, eval_s_score

test_c_acc = []
ood_c_acc = []
test_s_acc = []
ood_s_acc = []
mus = []
test_c_pos_scores = []
test_c_neg_scores = []
test_s_pos_scores = []
test_s_neg_scores = []
for t_idx in range(len(traces)):
    tr = traces[t_idx]
    meta = meta_info.iloc[t_idx]
    adapt_tr, test_tr, ood_tr = tr
    init_enr_idx = meta_info.iloc[t_idx].enr_idxs
    init_enr_embeds = sv_embeds[init_enr_idx]

    adapt_trial_idxs, adapt_scores, adapt_label, adapt_pred, adapt_enroll_pred = adapt_tr
    adapt_embeds = sv_embeds[adapt_trial_idxs.astype(np.int64)]
    adapt_enr_embeds = adapt_embeds[adapt_enroll_pred == 1]
    total_enr_embeds = np.concatenate([init_enr_embeds, adapt_enr_embeds])

    test_trial_idxs, test_scores, test_label, test_pred = test_tr
    test_embeds = sv_embeds[test_trial_idxs.astype(np.int64)]
    test_label = torch.from_numpy(test_label).byte()
    
    ood_trial_idxs, ood_scores, ood_label, ood_pred = ood_tr
    ood_embeds = sv_embeds[ood_trial_idxs.astype(np.int64)]
    ood_label = torch.from_numpy(ood_label).byte()
    
    enr_embeds = total_enr_embeds
#     enr_embeds = init_enr_embeds
    
#     imposter_embeds = adapt_embeds[adapt_pred==0]
    imposter_embeds = ood_imposter_embeds

    test_c_score, test_c_pred_acc, test_c_eer = eval_cosine_score(
                                             enr_embeds, test_embeds, 
                                             test_label, threshold)
    test_c_pos_scores += test_c_score[test_label == 1].numpy().tolist()
    test_c_neg_scores += test_c_score[test_label == 0].numpy().tolist()
    
    ood_c_score, ood_c_pred_acc, ood_c_eer = eval_cosine_score(
                                             enr_embeds, ood_embeds, 
                                             ood_label, threshold)
    
    z_score, z_pred_acc, z_eer = eval_z_score(
                                              enr_embeds, test_embeds, 
                                              test_label, threshold, 
                                              imposter_embeds)
    
    test_s_eer, test_s_pred_err, test_s_score, test_s_mu = eval_s_score1(
                                              enr_embeds, test_embeds,
                                              test_label, threshold,
                                              imposter_embeds)
    test_s_pos_scores += test_s_score[test_label == 1].numpy().tolist()
    test_s_neg_scores += test_s_score[test_label == 0].numpy().tolist()
   
    ood_s_eer, ood_s_pred_err, ood_s_score, ood_s_mu = eval_s_score1(
                                              enr_embeds, ood_embeds,
                                              ood_label, threshold,
                                              imposter_embeds)
    
        
    test_c_acc += [test_c_pred_acc]
    ood_c_acc += [ood_c_pred_acc]
    test_s_acc += [1 - test_s_pred_err]
    ood_s_acc += [1 - ood_s_pred_err]
    mus += [test_s_mu, ood_s_mu]


/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [41]:
from adaptive_sv_system.score_norm_utils import compute_eer
compute_eer(test_c_pos_scores, test_c_neg_scores)

(0.035660320338470886, 0.5882129073143005)

In [43]:
from adaptive_sv_system.score_norm_utils import compute_eer
compute_eer(test_s_pos_scores, test_s_neg_scores)

(0.02931399214264128, 2.32519268989563)

In [23]:
print("test_c_acc:{:.4f}\nood_c_acc:{:.4f}\ntest_s_acc:{:.4f}\nood_s_acc:{:.4f}".format(
    np.mean(test_c_acc), np.mean(ood_c_acc), np.mean(test_s_acc), np.mean(ood_s_acc)))

test_c_acc:0.9632
ood_c_acc:0.9967
test_s_acc:0.8295
ood_s_acc:0.6653


In [629]:
print("test_c_acc:{:.4f}\nood_c_acc:{:.4f}\ntest_s_acc:{:.4f}\nood_s_acc:{:.4f}".format(
    np.mean(test_c_acc), np.mean(ood_c_acc), np.mean(test_s_acc), np.mean(ood_s_acc)))

test_c_acc:0.9632
ood_c_acc:0.9967
test_s_acc:0.9099
ood_s_acc:1.0000
